In [1]:
### PFX prototyping

In [2]:
import pandas as pd
import os
from readability import Readability
import textstat
from openai import OpenAI
import json
import re
from pfx_fewshots import baseline_fewshot_prompt,example,baseline_fewshot_icd10_labeling_prompt,icd10_example,baseline_zeroshot_prompt
client = OpenAI()
#os.chdir(os.path.expanduser("../"))

In [3]:
df_fewshot = pd.read_csv('pfx_fewshot_examples_college.csv')

In [4]:
df_fewshot.head()


,Body_Part,Incidental_Finding,ICD10_code,PFx,PFx_ICD10_code
0,Head,Cerebral cavernous malformation,I67.1,A small cavernous malformation of the brain is...,I67.1
1,Head,Infundibulum of cerebral artery,Q28.2,An infundibulum detected on an MRI of the brai...,Q28.2
2,Neck,Tornwaldt cyst,J39.2,"A Tornwaldt cyst, often detected incidentally ...",J39.2
3,Head,Fluid in the right petrous apex,H70.8,A small amount of incidentally found fluid in ...,H70.8
4,Abdomen,Spinal hemangioma,D18.09,"A spinal hemangioma, often found incidentally ...",D18.09


In [5]:
df_eval = pd.read_csv('pfx_evaluation_data.csv')

In [6]:
df_eval.head()

,Body_Part,Incidental_Finding,ICD10_code,Unnamed: 3
0,Head,White matter lesions,R90.82,NaN
1,Head,Arachnoid cyst,Q04.3,NaN
2,Head,Pituitary microadenoma,D35.2,NaN
3,Head,Pineal cyst,Q04.6,NaN
4,Head,Chiari I malformation,Q07.0,NaN


In [7]:
print(icd10_example.format(**df_fewshot.iloc[0]))

<PFx>
A small cavernous malformation of the brain is a collection of abnormal blood vessels that is typically benign and often discovered incidentally when imaging is done. Patients should understand that these small malformations usually do not cause symptoms and usually do not require treatment. However, it's important to have periodic follow-ups with a neurologist to monitor for any changes, as there is a slight risk of bleeding which could lead to symptoms like headaches or seizures. If symptoms develop or if there are concerns about the malformation's behavior, the neurologist may discuss management options, which could range from observation to, in rare cases, surgical intervention.
</PFx>
<PFx_ICD10_code>
```{"PFx_ICD10_code":"I67.1"}```
</PFx_ICD10_code>




In [ ]:
def label_icd10s(pfx_outputs_json):
    pfx_icd10_fewshot_examples = ""
    for i,row in df_fewshot.iterrows():
        pfx_icd10_fewshot_examples += icd10_example.format(**row)
    
    pfx_icd10_codes = []
    for pfx_output in pfx_outputs_json:
        prompt = baseline_fewshot_icd10_labeling_prompt.format(examples=pfx_icd10_fewshot_examples,PFx=pfx_output['PFx'])
        pfx_icd10_response = client.chat.completions.create(
                #model="gpt-4o-mini",
                model="gpt-4o",
                temperature=0.0,
                messages=[
                    {"role": "system", "content": "You are an ICD10 medical coder for incidental findings." },
                    {"role": "user", "content": prompt }
                ],
                stream=False,
            )
        pfx_icd10_codes.append(pfx_icd10_response.choices[0])

In [8]:
def extract_json(openai_response):
    json_match = re.search(r'```.*(\{.*?\}).*```', openai_response.message.content, re.DOTALL)
    if json_match:
        json_str = json_match.group(1)
        # Convert the extracted string to a JSON object
        json_object = json.loads(json_str)
        #print(json_object)
        return json_object
    else:
        print("No JSON object found.") 
        return dict()

In [11]:
pfx_zeroshot_outputs = []
for i,row in df_eval.iloc[:].iterrows():
    #print(baseline_fewshot_prompt.format(examples=pfx_fewshot_examples,Incidental_Finding=row['Incidental_Finding']))
    prompt = baseline_zeroshot_prompt.format(Incidental_Finding=row['Incidental_Finding'])
    pfx_response = client.chat.completions.create(
        #model="gpt-4o-mini",
        model="gpt-4o",
        temperature=0.0,
        messages=[
            {"role": "system", "content": "You are a medical doctor rephrasing and explaining medical terminology to a patient." },
            {"role": "user", "content": prompt }
        ],
        stream=False,
    )
    
    pfx_zeroshot_outputs.append(pfx_response.choices[0])

In [18]:
pfx_zeroshot_outputs_json = list(map(extract_json,pfx_zeroshot_outputs))
pfx_zeroshot_outputs_icd10_labels = label_icd10s(pfx_zeroshot_outputs_json)
pfx_zeroshot_output_df = pd.DataFrame(pfx_zeroshot_outputs_json)

In [21]:
pfx_zeroshot_output_df['agent_icd10_codes'] = list(map(lambda x: list(x.values())[0],pfx_zeroshot_outputs_icd10_labels))

In [22]:
list(map(lambda x: x[0]==x[1],zip(pfx_zeroshot_output_df['ICD10_code'],pfx_zeroshot_output_df['agent_icd10_codes'])))

[False,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True]

In [23]:
#icd10 accuracy -- at least according to GPT4o
sum(_)/len(_)

0.8

In [29]:
pfx_fewshot_examples = ""
for i,row in df_fewshot.iterrows():
    pfx_fewshot_examples += example.format(**row)

    
pfx_fewshot_outputs = []
for i,row in df_eval.iloc[:].iterrows():
    #print(baseline_fewshot_prompt.format(examples=pfx_fewshot_examples,Incidental_Finding=row['Incidental_Finding']))
    prompt = baseline_fewshot_prompt.format(examples=pfx_fewshot_examples,Incidental_Finding=row['Incidental_Finding'])
    pfx_response = client.chat.completions.create(
        #model="gpt-4o-mini",
        model="gpt-4o",
        temperature=0.0,
        messages=[
            {"role": "system", "content": "You are a medical doctor rephrasing and explaining medical terminology to a patient." },
            {"role": "user", "content": prompt }
        ],
        stream=False,
    )
    
    pfx_fewshot_outputs.append(pfx_response.choices[0])

In [34]:
len(pfx_fewshot_outputs_json)

25

In [36]:
len(pfx_fewshot_outputs_icd10_labels)

25

In [30]:
pfx_fewshot_outputs_json = list(map(extract_json,pfx_fewshot_outputs))
pfx_fewshot_outputs_icd10_labels = label_icd10s(pfx_fewshot_outputs_json)


In [41]:
pfx_fewshot_outputs_icd10_labels

[{'PFx_ICD10_code': 'R90.82'},
 {'ICD10_code': 'Q04.6'},
 {'PFx_ICD10_code': 'D35.2'},
 {'PFx_ICD10_code': 'Q04.8'},
 {'ICD10_code': 'Q07.0'},
 {'PFx_ICD10_code': 'E04.1'},
 {'PFx_ICD10_code': 'R59.0'},
 {'ICD10_code': 'K11.8'},
 {'ICD10_code': 'I65.2'},
 {'ICD10_code': 'M50.20'},
 {'ICD10_code': 'R91.1'},
 {'PFx_ICD10_code': 'R59.0'},
 {'ICD10_code': 'K44.9'},
 {'ICD10_code': 'I25.10'},
 {'ICD10_code': 'I31.3'},
 {'PFx_ICD10_code': 'K76.8'},
 {'ICD10_code': 'N28.1'},
 {'PFx_ICD10_code': 'D35.00'},
 {'ICD10_code': 'K86.2'},
 {'ICD10_code': 'R16.1'},
 {'ICD10_code': 'N83.2'},
 {'ICD10_code': 'D25.9'},
 {'ICD10_code': 'N40.1'},
 {'ICD10_code': 'R59.1'},
 {'ICD10_code': 'N32.3'}]

In [44]:
 list(map(lambda x: list(x.values())[0],pfx_fewshot_outputs_icd10_labels))

['R90.82',
 'Q04.6',
 'D35.2',
 'Q04.8',
 'Q07.0',
 'E04.1',
 'R59.0',
 'K11.8',
 'I65.2',
 'M50.20',
 'R91.1',
 'R59.0',
 'K44.9',
 'I25.10',
 'I31.3',
 'K76.8',
 'N28.1',
 'D35.00',
 'K86.2',
 'R16.1',
 'N83.2',
 'D25.9',
 'N40.1',
 'R59.1',
 'N32.3']

In [46]:
pfx_fewshot_output_df = pd.DataFrame(pfx_fewshot_outputs_json)
pfx_fewshot_output_df['agent_icd10_codes'] = list(map(lambda x: list(x.values())[0],pfx_fewshot_outputs_icd10_labels))

#pfx_icd10_fewshot_examples = ""
#for i,row in df_fewshot.iterrows():
#    pfx_icd10_fewshot_examples += icd10_example.format(**row)
#    
#pfx_icd10_codes = []
#for pfx_output in pfx_outputs_json:
#    prompt = baseline_fewshot_icd10_labeling_prompt.format(examples=pfx_icd10_fewshot_examples,PFx=pfx_output['PFx'])
#    pfx_icd10_response = client.chat.completions.create(
#            #model="gpt-4o-mini",
#            model="gpt-4o",
#            temperature=0.0,
#            messages=[
#                {"role": "system", "content": "You are an ICD10 medical coder for incidental findings." },
#                {"role": "user", "content": prompt }
#            ],
#            stream=False,
#        )
#    pfx_icd10_codes.append(pfx_icd10_response.choices[0])
#    
#    
#

In [48]:
list(map(lambda x: x[0]==x[1],zip(pfx_fewshot_output_df['ICD10_code'],pfx_fewshot_output_df['agent_icd10_codes'])))

[True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True]

In [49]:
#icd10 accuracy -- at least according to GPT4o
sum(_)/len(_)

0.92

In [21]:
Readability(output_df.iloc[0]['PFx'])

In [22]:
_.flesch_kincaid()

ReadabilityException: 100 words required.

In [23]:
len(output_df.iloc[0]['PFx'].split())

98

In [24]:
import textstat

In [25]:
textstat.flesch_kincaid_grade(output_df.iloc[0]['PFx'])

14.0

In [26]:
textstat.flesch_reading_ease(output_df.iloc[0]['PFx'])

38.15